## A bit of help for the Twitter API

In [9]:
%pylab inline

import oauth2
import base64
import hashlib as hasher
import json

Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shape', 'box', 'Polygon']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [10]:
import folium
import geopandas as gpd
from shapely.geometry import *

### The preferred python package to use the oauth2 authentication protocol is


https://github.com/joestump/python-oauth2

### How to access twitter using the API

Go to https://developer.twitter.com/en/docs/basics/authentication/guides/single-user.html and see section *Using Python-OAuth2 library*

#### Authentication

First you need to create an account at twitter and 

In [58]:
# You will need a key and a secret
# I put it in a file in json format
twitter_credentials = json.load(open('keys.json', 'r'))['twitter1']
twitter_key = twitter_credentials['key']
twitter_secret = twitter_credentials['secret']
token_key = twitter_credentials['token_key']
token_secret = twitter_credentials['token_secret']

In [59]:
def oauth_req(url, key, secret, http_method="GET", post_body="", http_headers=None):
    consumer = oauth2.Consumer(key=twitter_key, secret=twitter_secret)
    token = oauth2.Token(key=key, secret=secret)
    client = oauth2.Client(consumer, token)
    resp, content = client.request( url, method=http_method, body=post_body.encode('utf-8'), headers=http_headers )
    return content

Returns a collection of the most recent Tweets posted by the authenticating user and the users they follow.

https://developer.twitter.com/en/docs/tweets/timelines/overview
    

In [60]:
home_timeline = oauth_req( 'https://api.twitter.com/1.1/statuses/home_timeline.json', 
                          key=token_key, secret=token_secret)
home_timeline

b'[]'

In [61]:
# Query settings
url="https://api.twitter.com/1.1/account/settings.json"
oauth_req( url, key=token_key, secret=token_secret)

b'{"time_zone":{"name":"Budapest","utc_offset":3600,"tzinfo_name":"Europe\\/Budapest"},"protected":false,"screen_name":"jegesm","always_use_https":true,"use_cookie_personalization":false,"sleep_time":{"enabled":false,"end_time":null,"start_time":null},"geo_enabled":false,"language":"en","discoverable_by_email":true,"discoverable_by_mobile_phone":true,"display_sensitive_media":false,"allow_contributor_request":"all","allow_dms_from":"following","allow_dm_groups_from":"following","translator_type":"none"}'

In [62]:
url="https://api.twitter.com/1.1/trends/place.json"
res = json.loads(oauth_req( url+"?id=1", key=token_key, secret=token_secret))


Extract geographical informations from the response

In [63]:
url="https://api.twitter.com/1.1/geo/id/df51dec6f4ee2b2c.json"
res=json.loads(oauth_req( url+"?id=1", key=token_key, secret=token_secret))
res

{'id': 'df51dec6f4ee2b2c',
 'url': 'https://api.twitter.com/1.1/geo/id/df51dec6f4ee2b2c.json',
 'place_type': 'neighborhood',
 'name': 'Presidio',
 'full_name': 'Presidio, San Francisco',
 'country_code': 'US',
 'country': 'United States',
 'contained_within': [{'id': '5a110d312052166f',
   'url': 'https://api.twitter.com/1.1/geo/id/5a110d312052166f.json',
   'place_type': 'city',
   'name': 'San Francisco',
   'full_name': 'San Francisco, CA',
   'country_code': 'US',
   'country': 'United States',
   'centroid': [-122.4461400159226, 37.759828999999996],
   'bounding_box': {'type': 'Polygon',
    'coordinates': [[[-122.514926, 37.708075],
      [-122.514926, 37.833238],
      [-122.357031, 37.833238],
      [-122.357031, 37.708075],
      [-122.514926, 37.708075]]]},
   'attributes': {}}],
 'geometry': None,
 'polylines': [],
 'centroid': [-122.46598425785236, 37.79989625],
 'bounding_box': {'type': 'Polygon',
  'coordinates': [[[-122.4891333, 37.786925],
    [-122.4891333, 37.8128675

In [64]:
ar=array(res['bounding_box']['coordinates'][0])
res['bounding_box']['coordinates'][0]=[ (b, a) for a,b in ar]
ar

array([[-122.4891333,   37.786925 ],
       [-122.4891333,   37.8128675],
       [-122.446306 ,   37.8128675],
       [-122.446306 ,   37.786925 ],
       [-122.4891333,   37.786925 ]])

In [65]:
url="https://api.twitter.com/1.1/users/suggestions.json"
res=json.loads(oauth_req( url+"?id=1", key=token_key, secret=token_secret))
res

[{'size': 16, 'slug': 'sports', 'name': 'Sports'},
 {'size': 13, 'slug': 'television', 'name': 'Television'},
 {'size': 15, 'slug': 'music', 'name': 'Music'},
 {'size': 15, 'slug': 'fashion', 'name': 'Fashion'},
 {'size': 14, 'slug': 'entertainment', 'name': 'Entertainment'},
 {'size': 12, 'slug': 'books', 'name': 'Books'},
 {'size': 15, 'slug': 'gaming', 'name': 'Gaming'},
 {'size': 9, 'slug': 'family', 'name': 'Family'},
 {'size': 17, 'slug': 'food-drink', 'name': 'Food & Drink'},
 {'size': 15, 'slug': 'funny', 'name': 'Funny'},
 {'size': 10, 'slug': 'business', 'name': 'Business'},
 {'size': 14, 'slug': 'government', 'name': 'Government'},
 {'size': 18, 'slug': 'news', 'name': 'News'}]

In [66]:
url="https://api.twitter.com/1.1/users/suggestions/sports/members.json"
res=json.loads(oauth_req( url+"?id=1", key=token_key, secret=token_secret))
res

[{'id': 300392950,
  'id_str': '300392950',
  'name': 'Twitter Sports',
  'screen_name': 'TwitterSports',
  'location': '',
  'description': 'Cheering with Tweets',
  'url': None,
  'entities': {'description': {'urls': []}},
  'protected': False,
  'followers_count': 15328809,
  'friends_count': 1577,
  'listed_count': 8795,
  'created_at': 'Tue May 17 17:54:29 +0000 2011',
  'favourites_count': 8175,
  'utc_offset': -25200,
  'time_zone': 'Pacific Time (US & Canada)',
  'geo_enabled': True,
  'verified': True,
  'statuses_count': 14435,
  'lang': 'en',
  'status': {'created_at': 'Thu Mar 21 16:09:12 +0000 2019',
   'id': 1108762491453087745,
   'id_str': '1108762491453087745',
   'text': 'Alright Bracketologists, #MarchMadness is officially in session.\nhttps://t.co/CCg2dKpwaD',
   'truncated': False,
   'entities': {'hashtags': [{'text': 'MarchMadness', 'indices': [25, 38]}],
    'symbols': [],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/CCg2dKpwaD',
      'expanded_u